# Test suite for micropython-ulab

In [20]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
import subprocess
import os

In [2]:
pwd

'/home/v923z/sandbox/micropython/v1.11/ulab/tests'

In [3]:
@magics_class
class PyboardMagic(Magics):
    @cell_magic
    @magic_arguments()
    @argument('-skip')
    @argument('-unix')
    @argument('-file')
    @argument('-data')
    @argument('-time')
    @argument('-memory')
    def micropython(self, line='', cell=None):
        args = parse_argstring(self.micropython, line)
        if args.skip: # doesn't care about the cell's content
            print('skipped execution')
            return None # do not parse the rest
        if args.unix: # tests the code on the unix port. Note that this works on unix only
            with open('/dev/shm/micropython.py', 'w') as fout:
                fout.write(cell)
            proc = subprocess.Popen(["../../micropython/ports/unix/micropython", "/dev/shm/micropython.py"], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(proc.stdout.read().decode("utf-8"))
            print(proc.stderr.read().decode("utf-8"))
            return None
        if args.file: # can be used to copy the cell content onto the pyboard's flash
            spaces = "    "
            try:
                with open(args.file, 'w') as fout:
                    fout.write(cell.replace('\t', spaces))
                    print('written cell to {}'.format(args.file))
            except:
                print('Failed to write to disc!')
            return None # do not parse the rest
        if args.data: # can be used to load data from the pyboard directly into kernel space
            message = pyb.exec(cell)
            if len(message) == 0:
                print('pyboard >>>')
            else:
                print(message.decode('utf-8'))
                # register new variable in user namespace
                self.shell.user_ns[args.data] = string_to_matrix(message.decode("utf-8"))
        
        if args.time: # measures the time of executions
            pyb.exec('import utime')
            message = pyb.exec('t = utime.ticks_us()\n' + cell + '\ndelta = utime.ticks_diff(utime.ticks_us(), t)' + 
                               "\nprint('execution time: {:d} us'.format(delta))")
            print(message.decode('utf-8'))
        
        if args.memory: # prints out memory information 
            message = pyb.exec('from micropython import mem_info\nprint(mem_info())\n')
            print("memory before execution:\n========================\n", message.decode('utf-8'))
            message = pyb.exec(cell)
            print(">>> ", message.decode('utf-8'))
            message = pyb.exec('print(mem_info())')
            print("memory after execution:\n========================\n", message.decode('utf-8'))

        else:
            message = pyb.exec(cell)
            print(message.decode('utf-8'))

ip = get_ipython()
ip.register_magics(PyboardMagic)

In [4]:
%%micropython -unix 1

import ulab as np

a = np.array(range(30), dtype=np.uint8)
b = a.reshape((2, 3, 5))
print(b)

array([[[0, 1, 2, 3, 4],
[5, 6, 7, 8, 9],
[10, 11, 12, 13, 14]],

[[15, 16, 17, 18, 19],
[20, 21, 22, 23, 24],
[25, 26, 27, 28, 29]]], dtype=uint8)




In [11]:
a = np.array(range(30), dtype=np.uint8)
b = a.reshape((2, 3, 5))
b

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14]],

       [[15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29]]], dtype=uint8)

# Vectorised calculations

In [39]:
%%micropython -unix 1

import ulab as np

a = np.array(range(10))
print(np.exp(a), '\n')

a = np.array(range(30))
b = a.reshape((2, 3, 5))
print(np.sin(b), '\n')

a = np.array(range(25))
b = a.reshape((5, 5))
c = b[2:5:2,1:5:2]
print(c.shape())

a = range(10)
print(np.exp(a))

array([1.0, 2.718281828459045, 7.38905609893065, 20.08553692318767, 54.59815003314424, 148.4131591025766, 403.4287934927351, 1096.633158428459, 2980.957987041728, 8103.083927575385], dtype=float) 

array([[[0.0, 0.8414709848078965, 0.9092974268256818, 0.1411200080598672, -0.7568024953079282],
[-0.9589242746631385, -0.2794154981989259, 0.6569865987187891, 0.9893582466233818, 0.4121184852417566],
[-0.5440211108893697, -0.9999902065507035, -0.5365729180004349, 0.4201670368266409, 0.9906073556948703]],

[[0.6502878401571168, -0.2879033166650653, -0.9613974918795568, -0.7509872467716761, 0.1498772096629524],
[0.9129452507276277, 0.836655638536056, -0.008851309290403876, -0.8462204041751706, -0.9055783620066238],
[-0.132351750097773, 0.7625584504796027, 0.956375928404503, 0.2709057883078691, -0.6636338842129676]]], dtype=float) 

(1, 2)
array([1.0, 2.718281828459045, 7.38905609893065, 20.08553692318767, 54.59815003314424, 148.4131591025766, 403.4287934927351, 1096.633158428459, 2980.95798704

In [35]:
a = np.array(range(10))
print(np.exp(a), '\n')

a = np.array(range(30))
b = a.reshape((2, 3, 5))
print(b.shape)
print(np.sin(b), '\n')

a = np.array(range(25))
b = a.reshape((5, 5))
print(np.sin(b[2:5:2,1:5:2]))

[1.00000000e+00 2.71828183e+00 7.38905610e+00 2.00855369e+01
 5.45981500e+01 1.48413159e+02 4.03428793e+02 1.09663316e+03
 2.98095799e+03 8.10308393e+03] 

(2, 3, 5)
[[[ 0.          0.84147098  0.90929743  0.14112001 -0.7568025 ]
  [-0.95892427 -0.2794155   0.6569866   0.98935825  0.41211849]
  [-0.54402111 -0.99999021 -0.53657292  0.42016704  0.99060736]]

 [[ 0.65028784 -0.28790332 -0.96139749 -0.75098725  0.14987721]
  [ 0.91294525  0.83665564 -0.00885131 -0.8462204  -0.90557836]
  [-0.13235175  0.76255845  0.95637593  0.27090579 -0.66363388]]] 

[[-0.99999021  0.42016704]
 [ 0.83665564 -0.8462204 ]]


In [21]:
sin(5.)

-0.9589242746631385

# Polynomial

In [20]:
%%micropython -unix 1

import ulab as np

a = np.array(range(10))
p = np.array([1, 2, 3])

print(np.polyval(p, a))

b = a.reshape((5, 2))
print(np.polyval(p, b))

array([3.0, 6.0, 11.0, 18.0, 27.0, 38.0, 51.00000000000001, 66.00000000000001, 83.00000000000001, 102.0], dtype=float)
array([[3.0, 6.0],
[11.0, 18.0],
[27.0, 38.0],
[51.00000000000001, 66.00000000000001],
[83.00000000000001, 102.0]], dtype=float)




In [16]:
a = np.array(range(10))
p = np.array([1, 2, 3])

print(np.polyval(p, a))

b = a.reshape((5, 2))
print(np.polyval(p, b))

[  3   6  11  18  27  38  51  66  83 102]
[[  3   6]
 [ 11  18]
 [ 27  38]
 [ 51  66]
 [ 83 102]]


In [28]:
%%micropython -unix 1

import ulab as np

x = np.array([0, 1, 2, 3, 4, 5, 6])
y = np.array([9, 4, 1, 0, 1, 4, 9])
print('independent values:\t', x)
print('dependent values:\t', y)
print('fitted values:\t\t', np.polyfit(x, y, 2))

independent values:	 array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0], dtype=float)
dependent values:	 array([9.0, 4.0, 1.0, 0.0, 1.0, 4.0, 9.0], dtype=float)
fitted values:		 array([1.0, -6.0, 9.000000000000004], dtype=float)




In [27]:
x = np.array([0, 1, 2, 3, 4, 5, 6])
y = np.array([9, 4, 1, 0, 1, 4, 9])
print('independent values:\t', x)
print('dependent values:\t', y)
print('fitted values:\t\t', np.polyfit(x, y, 2))

independent values:	 [0 1 2 3 4 5 6]
dependent values:	 [9 4 1 0 1 4 9]
fitted values:		 [ 1. -6.  9.]
